In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mean 
from numpy import linalg
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm

def reward(gaussian_region, characteristics):
    gaussian_region[0] = ((1 - LEARNING_RATE) * gaussian_region[0]) + (LEARNING_RATE * characteristics)
    euclidean_distance = np.linalg.norm(characteristics - gaussian_region[0])
    gaussian_region[1] = gaussian_region[1] + (LEARNING_RATE * euclidean_distance)

def penalty(gaussian_region, characteristics):
    gaussian_region[0] = ((1 + LEARNING_RATE) * gaussian_region[0]) + (LEARNING_RATE * characteristics)
    euclidean_distance = np.linalg.norm(characteristics - gaussian_region[0])
    gaussian_region[1] = gaussian_region[1] - (LEARNING_RATE * euclidean_distance)    
    
def gaussian_similarity(example, gaussian_region):
    euclidean_distance = np.linalg.norm(example - gaussian_region[0])
    return math.exp(-(pow(euclidean_distance, 2) / (2 * gaussian_region[1])))

data_frame = pd.read_csv('spambase_data.csv')
data_set = np.array(data_frame)
    
# A matrix that has all the spam emails.
first_category = data_set[0:1813, :]
#A matrix that has all the non-spam emails.
zero_category = data_set[1813:, :]

# Create the 10 sets that i need for 10 folds cross validation.
left_first = 0
right_first = 181
left_zero = 0
right_zero = 278
set1 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set2 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set3 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set4 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set5 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set6 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set7 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set8 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
right_first = right_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set9 = np.concatenate((first_category[left_first:right_first, :], zero_category[left_zero:right_zero, :]))

left_first = left_first + 181
left_zero = left_zero + 278
right_zero = right_zero + 278
set10 = np.concatenate((first_category[left_first:, :], zero_category[left_zero:, :]))

INIT_SPREADING = np.std(data_set[:, :-1]) * 0.1
LEARNING_RATE = 0.001
mean_1 = np.mean(first_category[:, :-1], axis = 0)
spreading_1 = np.std(first_category[:, :-1])
mean_0 = np.mean(zero_category[:, :-1], axis = 0)
spreading_0 = np.std(zero_category[:, :-1])
all_sets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]

print("\n---------------------------------------------------------------------------------------------------")
print("Variant LVQ Classifier")
all_accuracy = []
all_f1_score = []
for iteration1 in range(10):
    test_set = all_sets[iteration1]
    gaussian_regions = [[mean_1, spreading_1, 1], [mean_0, spreading_0, 0]]
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            training_set = all_sets[iteration2]
            for example in training_set: # Take a vector, where example[-1] is the category.
                characteristics = example[0:-1]
                category = example[-1]
                max_similarity = -1
                winner_gaussian_region_index = 0
                winner_gaussian_region = gaussian_regions[0]
                for gaussian_region_index in range(1, len(gaussian_regions)):
                    gaussian_region = gaussian_regions[gaussian_region_index]
                    similarity = gaussian_similarity(characteristics, gaussian_region)
                    if(similarity >= max_similarity):
                        winner_region = gaussian_region
                        winner_gaussian_region_index = gaussian_region_index
                        max_similarity = similarity
                if(category == gaussian_regions[winner_gaussian_region_index][2]):
                    reward(gaussian_regions[winner_gaussian_region_index], characteristics)
                else:
                    penalty(gaussian_regions[winner_gaussian_region_index], characteristics)
                    new_mean = characteristics
                    new_spreading = INIT_SPREADING
                    new_category = category
                    gaussian_regions.append([new_mean, new_spreading, new_category])
    for example in test_set:
        characteristics = example[0:-1]
        category = example[-1]
        max_similarity = -1
        winner_gaussian_region_index = 0
        winner_gaussian_region = gaussian_regions[0]
        for gaussian_region_index in range(1, len(gaussian_regions)):
            gaussian_region = gaussian_regions[gaussian_region_index]
            similarity = gaussian_similarity(characteristics, gaussian_region)
            if(similarity >= max_similarity):
                winner_region = gaussian_region
                winner_gaussian_region_index = gaussian_region_index
                max_similarity = similarity
        if(category == gaussian_regions[winner_gaussian_region_index][2]): # Positive
            if(category == 0):
                true_positive = true_positive + 1
            else:
                true_negative = true_negative + 1
        else: # Negative
            if(category == 0):
                false_positive = false_positive + 1
            else:
                false_negative = false_negative + 1           
    accuracy = (true_positive + true_negative) / test_set.shape[0] 
    precision = (true_positive) / (true_positive + false_positive)
    recall = (true_positive) / (true_positive + false_negative)
    f1_score = 2 * ((precision * recall) / (precision + recall))
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("End of Variant LVQ Classifier!")
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("K-NN Classifier")
number_of_neighbors = int(input("Give number of neighbors: "))
all_accuracy = []
all_f1_score = []
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    knn = KNeighborsClassifier(n_neighbors = number_of_neighbors)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("End of K-NN Classifier!")
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("Gaussian Naïve Bayes Classifier")
all_accuracy = []
all_f1_score = []
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score) 
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("End of Gaussian Naïve Bayes Classifier!")
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("MLP Stochastic Gradient Descent with one hidden layer.")
number_of_hidden_neurons = int(input("Give number of hidden neurons: "))
all_accuracy = []
all_f1_score = []
all_sets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    mlp = MLPClassifier(hidden_layer_sizes = (number_of_hidden_neurons), activation = 'logistic', max_iter = 800)
    mlp.fit(x_train, y_train)
    y_pred = mlp.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("\nMLP Stochastic Gradient Descent with two hidden layers.")
number_of_first_hidden_layer_neurons = int(input("Give number of first hidden layer neurons: "))
number_of_second_hidden_layer_neurons = int(input("Give number of second hidden layer neurons: "))
all_accuracy = []
all_f1_score = []
all_sets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    mlp = MLPClassifier(hidden_layer_sizes = (number_of_first_hidden_layer_neurons, number_of_second_hidden_layer_neurons), activation = 'logistic', max_iter = 800)
    mlp.fit(x_train, y_train)
    y_pred = mlp.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("SVM Classifier with linear kernel")
all_accuracy = []
all_f1_score = []
all_sets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]
number_of_neighbors = 3 # k = 3
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    svm_object = svm.LinearSVC()
    svm_object.fit(x_train, y_train)
    y_pred = svm_object.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("---------------------------------------------------------------------------------------------------")

print("\n---------------------------------------------------------------------------------------------------")
print("\nSVM Classifier with gaussian kernel")
all_accuracy = []
all_f1_score = []
all_sets = [set1, set2, set3, set4, set5, set6, set7, set8, set9, set10]
number_of_neighbors = 3
for iteration1 in range(10):
    all_test_set = all_sets[iteration1]
    all_training_set = None
    counter = 0
    for iteration2 in range(10):
        if(iteration2 != iteration1):
            if(counter == 0):
                counter = counter + 1
                all_training_set = all_sets[iteration2]
            else:
                all_training_set = np.concatenate((all_training_set, all_sets[iteration2]))
    x_test = all_test_set[:, :-1] # All the examples of the test set.
    y_test = all_test_set[:, -1] # The categories of the test set.
    x_train = all_training_set[:, :-1] # All the examples of the training set.
    y_train = all_training_set[:, -1] # The categories of the training set.
    svm_object = svm.SVC(gamma = 'auto', tol = 0.001)
    svm_object.fit(x_train, y_train)
    y_pred = svm_object.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred, average = 'weighted')
    all_accuracy.append(accuracy)
    all_f1_score.append(f1_score)
average_accuracy = mean(all_accuracy)
average_f1_score = mean(all_f1_score)
print("Average accuracy: ", average_accuracy)
print("Average F1 score: ", average_f1_score)
print("---------------------------------------------------------------------------------------------------")


---------------------------------------------------------------------------------------------------
Variant LVQ Classifier
Average accuracy:  0.680220646178093
Average F1 score:  0.7329336738363754
End of Variant LVQ Classifier!
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
K-NN Classifier
Give number of neighbors: 3
Average accuracy:  0.7842873035739119
Average F1 score:  0.7838844115566999
End of K-NN Classifier!
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Gaussian Naïve Bayes Classifier
Average accuracy:  0.8220442219440968
Average F1 score:  0.8222570673233353
End of Gaussian Naïve Bayes Classifier!
----------------------------------------------------------------------------

/home/nikolaossintoris/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/nikolaossintoris/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/nikolaossintoris/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/nikolaossintoris/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/nikolaossintoris/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblin

Average accuracy:  0.8254623835349743
Average F1 score:  0.8129534992518814
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------

SVM Classifier with gaussian kernel
Average accuracy:  0.8144277569183702
Average F1 score:  0.8151588780591539
---------------------------------------------------------------------------------------------------
